**Exercise 1: (5 points) If a decision tree is under-fitting the training dataset, is it a good idea to try scaling
the input features?**

No, because there is no need for scaling in decision tree models because it does not affect the output. It would not solve the problem of over fitting.

**Exercise 2: (5 points) If a decision tree is over-fitting the training dataset, is it a good idea to try decreasing
max depth?**

Yes, reducing the max depth will help regularize the model and reduce the risk of overfitting.

**Exercise 3: (4 points) Why would you use a random forest instead of a decision tree?**
- **(a) For a lower training error.**
- **(b) To reduce the variance of the model.**
- **(c) For a model that it is easier for human to interpret.**
- **(d) (a) and (b)**
- **(e) (a) and (c)**
- **(f) (b) and (c)**
- **(g) All of the above**
- **(h) None of the above**

(d) a and b

It reduces error and variance. However, it dones not become easier to interpret because it is multiple decision trees

**Exercise 4: (4 points) Which of the following is/are TRUE about bagging trees?**
- **(a) In bagging trees, the trees are grown independent of each other.**
- **(b) In bagging trees, the trees are grown in sequence.**
- **(c) Bagging is a method for improving the performance by aggregating the results of weak learners.**
- **(d) (a) and (c)**
- **(e) (b) and (c)**
- **(f) None of the above.**

(d) a and c

In bagging, trees are grown independently and bagging is a method for improving the performance by aggregating the results of weak learners.
(note: boosting involves growing trees sequentially.)

**Exercise 5: (12 points) Suppose you are building random forest model, which split a node on the attribute,
that has highest information gain (using the Gini index). In the below image, which attribute
which has the highest information gain? Show all your calculations.**

You would choose "outlook" because it had the lowest weighted sum of Gini indices (roughly 0.34) and therefore, it offers the highest information gain.

refer to pdf submission for calculations

**Exercise 6: Consider the framingham.csv data file. The dataset is publically available on the Kaggle website, and it is from an ongoing cardiovascular study on residents of the town of Framingham, Massachusetts. The classification goal is to predict whether the patient has 10-year risk of future coronary heart disease (CHD). The dataset provides the patients? information. It includes over 4,000 records and 15 attributes. Each attribute is a potential risk factor. There are both demographic, behavioral and medical risk factors. In Python, answer the following:**

(a) (4 points) Load the data file to you S3 bucket. Using the pandas library, read the csv data
file and create a data-frame called heart.

In [1]:
import boto3
import pandas as pd; pd.set_option('display.max_column', 100)
import numpy as np
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")


from sklearn.model_selection import train_test_split
from sklearn.metrics import recall_score
from sklearn.ensemble import RandomForestClassifier


## Defining the s3 bucket
s3 = boto3.resource('s3')
bucket_name = 'craig-shaffer-data-445-bucket'
bucket = s3.Bucket(bucket_name)

## Defining the file to be read from s3 bucket
file_key = 'framingham.csv'

bucket_object = bucket.Object(file_key)
file_object = bucket_object.get()
file_content_stream = file_object.get('Body')

# reading the datafile
heart = pd.read_csv(file_content_stream)
heart.head()

,male,age,education,currentSmoker,cigsPerDay,BPMeds,prevalentStroke,prevalentHyp,diabetes,totChol,sysBP,diaBP,BMI,heartRate,glucose,TenYearCHD
0,1,39,4.0,0,0.0,0.0,0,0,0,195.0,106.0,70.0,26.97,80.0,77.0,0
1,0,46,2.0,0,0.0,0.0,0,0,0,250.0,121.0,81.0,28.73,95.0,76.0,0
2,1,48,1.0,1,20.0,0.0,0,0,0,245.0,127.5,80.0,25.34,75.0,70.0,0
3,0,61,3.0,1,30.0,0.0,0,1,0,225.0,150.0,95.0,28.58,65.0,103.0,1
4,0,46,3.0,1,23.0,0.0,0,0,0,285.0,130.0,84.0,23.10,85.0,85.0,0


(b) (3 points) Remove observations with missing values.

In [2]:
#dropping observations w/ missing values
heart=heart.dropna()

(c) (30 points) Using all the available variables as the predictor variables, and TenYearCHD as
the target variable, do the following:
- (i) Split the data into train (80%) and test (20%) (taking into account the proportion of 0s and 1s).
- (ii) Using the train data-frame, build a random forest classifier (using 500 trees).
- (iii) Extract the feature importance of each of the variables.

Repeat (i)-(iii) 100 times. Compute the average importance of each of the variables across
the 100 splits. After that, select the top 5 variables (the ones with top 5 average importance)
as the predictor variables.


In [15]:
importance = list()

for i in range (0,100):

    # defining the input and target variables
    x = heart.drop(columns = ['TenYearCHD'],axis=1)
    y = heart['TenYearCHD']

    #splitting the data
    x_train,x_test,y_train,y_test = train_test_split(x,y,test_size = .2,stratify=y)

    #build the model
    RF_md = RandomForestClassifier(n_estimators = 500).fit(x_train,y_train)

    #extract the feature importances
    importance.append(RF_md.feature_importances_)

In [17]:
importance = pd.DataFrame(importances)
importance.columns = ['male','age','education','currentSmoker','cigsPerDay','BPMeds','prevalentStroke','prevalentHyp','diabetes','totChol','sysBP','diaBP','BMI','heartRate','glucose']
i = pd.DataFrame({'Importance':importance.apply(np.mean, axis = 0)})
i.sort_values(by = 'Importance', ascending = False)

,Importance
sysBP,0.134887
BMI,0.127056
age,0.124192
totChol,0.122082
glucose,0.120016
diaBP,0.118947
heartRate,0.096322
cigsPerDay,0.050477
education,0.036895
male,0.021208


    The top 5 variables are sysBP, BMI, age, totChol, and glucose

(d) (35 points) Using the top 5 variables from part (c) as the predictor variables and TenYearCHD as the target variable, do the following:
- (i) Split the data into train (80%) and test (20%) (taking into account the proportion of 0s and 1s).
- (ii) Using the train data-frame, build a random forest classifier (using 500 trees and maximum depth tree equal to 3). Using this model, predict the likelihood of risk of coronary disease of the patients in the test data-frame. Using 10% as cutoff value, report the recall.
- (iii) Using the train data-frame, build a random forest classifier (using 500 trees and maximum depth tree equal to 5). Using this model, predict the likelihood of risk of coronary disease of the patients in the test data-frame. Using 10% as cutoff value, report the recall.
- (iv) Using the train data-frame, build a random forest classifier (using 500 trees and maximum depth tree equal to 7). Using this model, predict the likelihood of risk of coronary disease of the patients in the test data-frame. Using 10% as cutoff value, report the recall.

Repeat (i)-(iii) 100 times. Compute the average recall of each of the models across the 100
iterations. What model would use to predict TenYearCHD? Be specific.


In [20]:
#lists to store recall values for each model
md1_results= list()
md2_results= list()
md3_results= list()

#defining the input (top 5 features) and target for RF
x = heart[['sysBP', 'BMI', 'age', 'totChol', 'glucose']]
y = heart['TenYearCHD']
    
for i in range (0,100):

    #splitting the data
    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size= 0.2, stratify=y)

    
    #RF model with max_depth = 3
    RF_md1 = RandomForestClassifier(max_depth = 3, n_estimators = 500).fit(x_train, y_train)
    
    #predict on test
    RF_pred1 = RF_md1.predict_proba(x_test)[:,1]
    
    #changing likelihoods to labels w/ 10% cutoff
    RF_label1 = np.where(RF_pred1 < .1, 0, 1)
    
    #computing the recall
    RF_recall1 = recall_score(y_test, RF_label1)
    md1_results.append(RF_recall1)

    

    #RF model with max_depth = 5
    RF_md2 = RandomForestClassifier(max_depth = 5, n_estimators = 500).fit(x_train, y_train)
    
    #predict on test
    RF_pred2 = RF_md2.predict_proba(x_test)[:,1]
    
    #changing likelihoods to labels w/ 10% cutoff
    RF_label2 = np.where(RF_pred2 < .1, 0, 1)
    
    #computing the recall
    RF_recall2 = recall_score(y_test, RF_label2)
    md2_results.append(RF_recall2)

    

    #RF model with max_depth = 7
    RF_md3 = RandomForestClassifier(max_depth = 7, n_estimators = 500).fit(x_train, y_train)
    
    #predict on test
    RF_pred3 = RF_md3.predict_proba(x_test)[:,1]

    #changing likelihoods to labels w/ 10% cutoff
    RF_label3 = np.where(RF_pred3 < .1, 0, 1)
    
    #computing the recall
    RF_recall3 = recall_score(y_test, RF_label3)
    md3_results.append(RF_recall3)

In [21]:
print('the avg recall for model 1 (max_depth=3):', np.mean(md1_results))
print('the avg recall for model 2 (max_depth=5):',np.mean(md2_results))
print('the avg recall for model 3 (max_depth=7):',np.mean(md3_results))

the avg recall for model 1 (max_depth=3): 0.8414285714285715
the avg recall for model 2 (max_depth=5): 0.8230357142857143
the avg recall for model 3 (max_depth=7): 0.8095535714285714


Based on the results above, I would use the first model (max_depth=3) because it had the highest avg recall.